In [7]:
#Question-6(c)
import numpy as np
import pandas as pd
import math

input_data = pd.read_csv('https://raw.githubusercontent.com/ParthKataria/OperatingSystem/master/Housing%20Price%20data%20set.csv')

Price = input_data['price']
FloorArea = input_data['lotsize']
NoOfBedrooms = input_data['bedrooms']
NoOfBathrooms = input_data['bathrms']

# Performing feature scanning on FloorArea
FloorArea_Mean = np.mean(FloorArea)
FloorArea_Max = max(FloorArea)
FloorArea_Min = min(FloorArea)
FloorArea_Scaled = []
for i in FloorArea:
	FloorArea_Scaled.append((i - FloorArea_Mean) / (FloorArea_Max - FloorArea_Min))

#segmenting the features
FeaturesTrain = []
for i in range(383):
	FeaturesTrain.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
PriceTrain = Price[:383]
PriceTest = []
FeaturesTest = []
for i in range(383, len(Price)):
	FeaturesTest.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
	PriceTest.append(Price[i])
m = len(FeaturesTrain)

# Function to calculate Slope to find coefficients
def Slope(Coeff, FeaturesTrain, PriceTrain, ind):
	Error = 0
	for i in range(len(FeaturesTrain)):
		itr = 0
		for j in range(len(Coeff)):
			itr = itr + Coeff[j] * FeaturesTrain[i][j]
		Error += (itr - PriceTrain[i]) * FeaturesTrain[i][ind]
	return Error

# Using scaled batch gradient without regularisation
print("Using scaled batch gradient without regularisation")
LearningRate = 0.001
m = len(FeaturesTrain)

Coeff = [0, 0, 0, 0]
print("Initial coefficients: ")
print(Coeff)
for i in range(5000):
	TempCoeff = Coeff.copy()
	for j in range(len(Coeff)):
		TempCoeff[j] = TempCoeff[j] - ((LearningRate / m) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))
	Coeff = TempCoeff.copy()
print("Final coefficients are:")
print(Coeff)

# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 90
print("Mean absolute percentage error is : " + str(Error))
print()

# Using scaled batch gradient with regularisation
print("Using scaled batch gradient with regularisation")
LearningRate = 0.001
LambdaParameter = -49
Coeff = [0, 0, 0, 0]
print("Initial coefficients: ")
print(Coeff)
for epochs in range(5000):
	TempCoeff = Coeff.copy()
	for j in range(len(Coeff)):
		if (j == 0):
			TempCoeff[j] = TempCoeff[j] - ((LearningRate / m) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))	
		else:
			TempCoeff[j] = (1 - LearningRate * LambdaParameter / m) * TempCoeff[j] - ((LearningRate / m) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))
	Coeff = TempCoeff.copy()
print("Final coefficients are:")
print(Coeff)

# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

def SlopeStoch(Coeff,FeaturesTrain,ActualVal,ind):
	itr = 0
	for j in range(len(Coeff)):
		itr = itr + Coeff[j]*FeaturesTrain[j]
	return (itr - ActualVal) * FeaturesTrain[ind]

# Using Scaled Stochastic gradient without regularisation.
print("Using Stochastic gradient without regularisation")

LearningRate = 0.005
Coeff = [0, 0, 0, 0]
print("Initial coefficients: ")
print(Coeff)

for iter in range(10):
	for i in range(len(PriceTrain)):
		TempCoeff = Coeff.copy()
		for j in range(4):
			TempCoeff[j] = TempCoeff[j] - (LearningRate * (SlopeStoch(Coeff, FeaturesTrain[i], PriceTrain[i], j)))
		Coeff = TempCoeff.copy()

print("Final coefficients are:")
print(Coeff)

# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

# Using Scaled Minibatch gradient without regularisation for batch size = 20
print("Using Scaled Minibatch gradient without regularisation for batch size = 20")

BatchSize = 20;
LearningRate = 0.002
Coeff = [0, 0, 0, 0]
NoOfBatches = math.ceil(len(PriceTrain) / BatchSize)
equallyDiv = False
if (len(PriceTrain) % BatchSize == 0):
	equallyDiv = True;

for epoch in range(30):
	for batch in range(NoOfBatches):
		Summation = [0, 0, 0, 0]
		for j in range(len(Coeff)):
			for i in range(BatchSize):
				if (batch * BatchSize + i == len(FeaturesTrain)):
					break
				PredictedValue = 0.0
				for wj in range(len(Coeff)):
					PredictedValue += Coeff[wj] * FeaturesTrain[batch * BatchSize + i][wj]
				PredictedValue -= PriceTrain[batch * BatchSize + i]
				PredictedValue *= FeaturesTrain[batch * BatchSize + i][j]
				Summation[j] += PredictedValue;

		if (not equallyDiv and batch == NoOfBatches - 1):
			for j in range(len(Summation)):
				Coeff[j] -= (Summation[j] / (len(PriceTrain) % BatchSize)) * LearningRate
		else:
			for j in range(len(Summation)):
				Coeff[j] -= (Summation[j] / BatchSize) * LearningRate
print("Final coefficients are:")
print(Coeff)

# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

# Using Scaled Minibatch gradient with regularisation for batch size = 20
print("Using Scaled Minibatch gradient with regularisation for batch size = 20")

BatchSize = 20;
LearningRate = 0.002
LambdaParameter = -372
Coeff = [0, 0, 0, 0]
NoOfBatches = math.ceil(len(PriceTrain) / BatchSize)
equallyDiv = False
if (len(PriceTrain) % BatchSize == 0):
	equallyDiv = True;

for epoch in range(30):
	for batch in range(NoOfBatches):
		Summation = [0, 0, 0, 0]
		for j in range(len(Coeff)):
			for i in range(BatchSize):
				if (batch * BatchSize + i == len(FeaturesTrain)):
					break
				PredictedValue = 0.0
				for wj in range(len(Coeff)):
					PredictedValue += Coeff[wj] * FeaturesTrain[batch * BatchSize + i][wj]
				PredictedValue -= PriceTrain[batch * BatchSize + i]
				PredictedValue *= FeaturesTrain[batch * BatchSize + i][j]
				Summation[j] += PredictedValue;

		if (not equallyDiv and batch == NoOfBatches - 1):
			for j in range(len(Summation)):
				if j == 0:
					Coeff[j] -= (Summation[j] / (len(PriceTrain) % BatchSize)) * LearningRate
				else:
					Coeff[j] = (1 - LearningRate * LambdaParameter / m) * Coeff[j] - (Summation[j] / (len(PriceTrain) % BatchSize)) * LearningRate
		else:
			for j in range(len(Summation)):
				if j == 0:
					Coeff[j] -= (Summation[j] / BatchSize) * LearningRate
				else:
					Coeff[j] = (1 - LearningRate * LambdaParameter / m) * Coeff[j] - (Summation[j] / BatchSize) * LearningRate
print("Final coefficients are:")
print(Coeff)

# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

Using scaled batch gradient without regularisation
Initial coefficients: 
[0, 0, 0, 0]
Final coefficients are:
[7730.872053867435, 8069.264303687423, 11077.015405893277, 18485.569122447192]
Mean absolute percentage error is : 18.30641888247704

Using scaled batch gradient with regularisation
Initial coefficients: 
[0, 0, 0, 0]
Final coefficients are:
[5037.585668619078, 11147.667574879839, 10378.580439168689, 22647.298983883848]
Mean absolute percentage error is : 19.92701396456417

Using Stochastic gradient without regularisation
Initial coefficients: 
[0, 0, 0, 0]
Final coefficients are:
[18648.663069990776, 15073.501985961251, 15766.862790309351, 22357.23427068568]
Mean absolute percentage error is : 32.87903970192347

Using Scaled Minibatch gradient without regularisation for batch size = 20
Final coefficients are:
[6340.552295015755, 2827.87646125345, 15916.90159915714, 10968.772912396124]
Mean absolute percentage error is : 20.275262089497147

Using Scaled Minibatch gradient with

In [ ]:
#Question-6(d)
import numpy as np
import pandas as pd
import math
def cal_err(Y_train, Y_pred):
    error = 0
    for i in range(len(Y_train)):
        error += abs(Y_train[i] - Y_pred[i]) / Y_train[i]
    error = error / len(Y_train)
    return error * 100

def kernel(X_train, xi, Tau):
    return np.exp(-np.sum((xi - X_train) ** 2, axis = 1) / (2 * Tau * Tau))

def LocallyWeightedLR(X_train, xi, Y_train, Tau):
	XT = np.transpose(X_train)
	W = kernel(X_train, xi, Tau)
	MXTW = XT * W
	MXTWX = np.matmul(MXTW, X_train)
	InverseMXTWX = np.linalg.pinv(MXTWX)
	InverseMXTWXMXTW = np.matmul(InverseMXTWX, MXTW)
	InverseMXTWXMXTWY = np.matmul(InverseMXTWXMXTW, Y_train)
	InverseMXTWXMXTWYTranspose = np.transpose(InverseMXTWXMXTWY)
	return InverseMXTWXMXTWYTranspose.dot(xi)

In [ ]:
input_data = pd.read_csv('https://raw.githubusercontent.com/ParthKataria/OperatingSystem/master/Housing%20Price%20data%20set.csv', usecols = ["price", "lotsize", "bedrooms", "bathrms"])
FloorArea = input_data['lotsize']
NoOfBedrooms = input_data['bedrooms']
NoOfBathrooms = input_data['bathrms']
Y_train = input_data['price']
Y_train = np.array(Y_train)
Y_train = Y_train.reshape(Y_train.shape[0], 1)

# Performing feature scanning on FloorArea
FloorArea_Mean = np.mean(FloorArea)
FloorArea_Max = max(FloorArea)
FloorArea_Min = min(FloorArea)
FloorArea_Scaled = []
for i in FloorArea:
	FloorArea_Scaled.append((i - FloorArea_Mean) / (FloorArea_Max - FloorArea_Min))

X_train = []
for i in range(len(FloorArea)):
	X_train.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
X_train = np.array(X_train)

Tau = 0.00005
print("Using Locally Weighted Linear Regression for Tau = " + str(Tau))
pred = []
for i in range(X_train.shape[0]):
	y_pred = LocallyWeightedLR(X_train, X_train[i], Y_train, Tau)
	pred.append(y_pred)
print("Mean absolute percentage error is : " + str(cal_err(Y_train,pred)))
print()

Price = input_data['price']

#segmenting the features
FeaturesTrain = []
for i in range(383):
	FeaturesTrain.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
PriceTrain = Price[:383]
PriceTest = []
FeaturesTest = []
for i in range(383, len(Price)):
	FeaturesTest.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
	PriceTest.append(Price[i])
m = len(FeaturesTrain)

Using Locally Weighted Linear Regression for Tau = 5e-05
Mean absolute percentage error is : [5.40732082]



In [ ]:
# Function to calculate Slope to find coefficients
def Slope(Coeff, FeaturesTrain, PriceTrain, ind):
	Error = 0
	for i in range(len(FeaturesTrain)):
		itr = 0
		for j in range(len(Coeff)):
			itr = itr + Coeff[j] * FeaturesTrain[i][j]
		Error += (itr - PriceTrain[i]) * FeaturesTrain[i][ind]
	return Error

# Using scaled batch gradient with regularisation
print("Using scaled batch gradient with regularisation")
LearningRate = 0.001
LambdaParameter = -49
Coeff = [0, 0, 0, 0]
print("Initial coefficients: ")
print(Coeff)
for epochs in range(5000):
	TempCoeff = Coeff.copy()
	for j in range(len(Coeff)):
		if (j == 0):
			TempCoeff[j] = TempCoeff[j] - ((LearningRate / m) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))	
		else:
			TempCoeff[j] = (1 - LearningRate * LambdaParameter / m) * TempCoeff[j] - ((LearningRate / m) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))
	Coeff = TempCoeff.copy()
print("Final coefficients are:")
print(Coeff)

# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

def SlopeStoch(Coeff,FeaturesTrain,ActualVal,ind):
	itr = 0
	for j in range(len(Coeff)):
		itr = itr + Coeff[j]*FeaturesTrain[j]
	return (itr - ActualVal) * FeaturesTrain[ind]

Using scaled batch gradient with regularisation
Initial coefficients: 
[0, 0, 0, 0]
Final coefficients are:
[5037.585668619078, 11147.667574879839, 10378.580439168689, 22647.298983883848]
Mean absolute percentage error is : 19.92701396456417



In [ ]:
# Using Scaled Stochastic gradient with regularisation.
print("Using Stochastic gradient with regularisation")

# I tried with different values of tau but found this to be the best.
LearningRate = 0.001
LambdaParameter = 142000
Coeff = [0, 0, 0, 0]
print("Initial coefficients: ")
print(Coeff)

for iter in range(10):
	for i in range(len(PriceTrain)):
		TempCoeff = Coeff.copy()
		for j in range(4):
			if j == 0:
				TempCoeff[j] = TempCoeff[j] - (LearningRate * (SlopeStoch(Coeff, FeaturesTrain[i], PriceTrain[i], j)))
			else:
				TempCoeff[j] = (1 - LearningRate * LambdaParameter / m) * TempCoeff[j] - (LearningRate * (SlopeStoch(Coeff, FeaturesTrain[i], PriceTrain[i], j)))
		Coeff = TempCoeff.copy()

print("Final coefficients are:")
print(Coeff)

# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

# Using Scaled Minibatch gradient with regularisation for batch size = 30
print("Using Scaled Minibatch gradient with regularisation for batch size = 30")

BatchSize = 30;
LearningRate = 0.002
LambdaParameter = -372
Coeff = [0, 0, 0, 0]
NoOfBatches = math.ceil(len(PriceTrain) / BatchSize)
equallyDiv = False
if (len(PriceTrain) % BatchSize == 0):
	equallyDiv = True;

for epoch in range(30):
	for batch in range(NoOfBatches):
		Summation = [0, 0, 0, 0]
		for j in range(len(Coeff)):
			for i in range(BatchSize):
				if (batch * BatchSize + i == len(FeaturesTrain)):
					break
				PredictedValue = 0.0
				for wj in range(len(Coeff)):
					PredictedValue += Coeff[wj] * FeaturesTrain[batch * BatchSize + i][wj]
				PredictedValue -= PriceTrain[batch * BatchSize + i]
				PredictedValue *= FeaturesTrain[batch * BatchSize + i][j]
				Summation[j] += PredictedValue;

		if (not equallyDiv and batch == NoOfBatches - 1):
			for j in range(len(Summation)):
				if j == 0:
					Coeff[j] -= (Summation[j] / (len(PriceTrain) % BatchSize)) * LearningRate
				else:
					Coeff[j] = (1 - LearningRate * LambdaParameter / m) * Coeff[j] - (Summation[j] / (len(PriceTrain) % BatchSize)) * LearningRate
		else:
			for j in range(len(Summation)):
				if j == 0:
					Coeff[j] -= (Summation[j] / BatchSize) * LearningRate
				else:
					Coeff[j] = (1 - LearningRate * LambdaParameter / m) * Coeff[j] - (Summation[j] / BatchSize) * LearningRate
print("Final coefficients are:")
print(Coeff)

# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

Using Stochastic gradient with regularisation
Initial coefficients: 
[0, 0, 0, 0]
Final coefficients are:
[63592.672255517704, 50.82794652908167, 490.4664942089522, 198.33597517414273]
Mean absolute percentage error is : 22.669921716639656

Using Scaled Minibatch gradient with regularisation for batch size = 30
Final coefficients are:
[2681.43806108953, 2059.662947401348, 17584.987500198524, 12538.251896070407]
Mean absolute percentage error is : 20.000670512331244

